# Initial instance configuration

This guide walks you through the initial steps to get started with a fresh installation of a KAWA server on premise.

It requires an API key for an ADMIN user. 

Please make sure to follow the installation steps for KYWY (https://github.com/kawa-analytics/kywy-documentation)


In [ ]:
from kywy.client.kawa_client import KawaClient as K

kawa = K.load_client_from_environment()
cmd = kawa.commands

## 1. Install the license

This allows to load a license file into a running kawa instance. License files are json files. If you need a license, please visit www.kawa.ai.



In [ ]:
kawa.commands.set_license(path_to_license_file='/tmp/path/to/license.json')

## 2. Change the setup admin password

In [ ]:
cmd.change_user_password('setup-admin@kawa.io', '********');

## 3. Activate the desired features

This will toggle the feature flags of the KAWA instance. The available flags are:

- list-principals (Allow to show all the users in user selection lists - like members of workspaces etc..)
- scripts (Enable script support)
- automations (Enable automation support)
- dashboard-export (Enable dashboard export)
- data-sample (Shows a list of samples in the datasource section)
- user-sign-up (When activated, users can signup on the login page with their email address)

In [ ]:
cmd.toggle_feature(feature_name='data-samples', is_enabled=True)
cmd.toggle_feature(feature_name='automations', is_enabled=True);
cmd.toggle_feature(feature_name='scripts', is_enabled=True);
cmd.toggle_feature(feature_name='user-sign-up', is_enabled=True);

## 4. Configure the workspace creation behaviour

This step lets you choose the behavious of workspaces creation.

__Please note that the server needs to be restarted for these parameters to be taken in account.__

In [ ]:
cmd.replace_configuration('WorkspaceConfiguration', {
    
    # If set to True, then only admins will be able to create new workspaces
    'workspaceCreationRestrictedToAdmins': True,
    
    # Two possible values: 
    # 'CREATE_NEW'  > KAWA will create a personal workspace for the user when they login for the first time
    # 'USE_DEFAULT' > KAWA will add all new users into 'defaultWorkspaceId'
    'workspaceInitializationMode': 'USE_DEFAULT', 
    
    # This is required if the workspaceInitializationMode is set to USE_DEFAULT
    'defaultWorkspaceId': '1', 
})

## 5. Configure the SMTP server (optional)

This step is required only if you want KAWA to be able to send emails through your company SMTP server.
The credentials of the SMTP server are configured during the installation process.

__Please note that the server needs to be restarted for the SMTP parameters to be taken in account.__

In [ ]:
# Turn on SMTP communications
# To turn back off, use: LOG instead of SMTP
cmd.replace_communication_provider_type('SMTP')

# Configure the SMTP server
# Use extraSmtpProperties to put any additional configuration from
#  https://javadoc.io/doc/com.sun.mail/jakarta.mail/latest/jakarta.mail/com/sun/mail/smtp/package-summary.html
cmd.replace_configuration('SmtpConfig', {
    'host': 'smtp.wayne.com',
    'port': 2525,
    'sslEnabled': False,
    'startTlsEnabled': True,
    'authenticationEnabled': True,
    'extraSmtpProperties': {
        "mail.smtp.ssl.trust": "some-domain.com"
    }
})

# Defines a subject + sender for email activation emails
cmd.replace_configuration('EmailActivatorConfiguration', {
    'sender': 'postmaster@kawa-analytics.com',
    'subject': 'One last step to activate your account'
})

# Defines a subject + sender for autimation emails
cmd.replace_configuration('AutomationBehaviourConfiguration', {
    'emailSender': 'robot@kawa-analytics.com',
    'emailSubject': 'New automation from KAWA'
});

## 6. Connect a Python script runner (optional)

Repeat this step for each workspace you wish to connect a python runner to.
It will allow users from this workspace to benefit from Python columns and Python automations.

In [ ]:
# Will activate a runner in this workspace
WORKSPACE_ID=1
kawa.set_active_workspace_id(workspace_id=WORKSPACE_ID)


cmd.add_script_runner(
    name='Main runner',
    host='kawa-script-runner',
    port=8815,
    tls=False
)

print(kawa.runner_health())

The below configuration lets you automatically connect a Python runner to each workspace. (Restart is necessary for this to be taken in account).
You can still configure your runners individually, which will override the below Global Runner configuration.

In [ ]:
cmd.replace_configuration('GlobalPythonRunnerConfig', {
    'name':'Main runner',
    'host':'kawa-script-runner',
    'port':8815,
    'tls':False,
    'enabled':True
})


## 7. Configure SSO (optional)

### 7.a Option 1: Open ID Connect (OIDC)

KAWA supports OIDC based SSO authentication. 
Make sure to have configured your application integration on the identity provider. 


Please refer to the documentation that matches your IDP:

- [OKTA](https://github.com/kawa-analytics/kawa-docker-install/blob/main/documentation/CONFIGURE_OIDC_WITH_OKTA.md)

__Please note that the server needs to be restarted for the Authentication parameters to be taken in account.__

In [ ]:
cmd.replace_configuration('OAuth2ClientConfiguration', {
    'clientId': '******', 
    'openidIssuer': 'https://{your subdomain}.okta.com/oauth2/default',
    'clientHost': 'https://{kawa url}', 
})

The following additional parameters are supported:

- *userIdField*: Specify which field will be used from the user info  to determine the user's unique id. The default value is `sub`. If you wish to load the unique id of the user from their email addresses, you can set it to `email`.

- *stateValidationEnabled*: Defaults to `true` (It is recommended to leave it to `true` for additional security)

- *emailMustBeVerified*: Defaults to `true`. Set it to `false` if you want to skip this verification. 

- *offlineAccessScope*: Defaults to `true`. Some implementations of OIDC require this to be set to true to work with refreshTokens (ie: OKTA). Some other implementation do not support it (ie: AWS COGNITO).

In [ ]:
# Below is an example working with AWS Cognito, disabling the email verification and offlineAccessScope scope.
cmd.replace_configuration('OAuth2ClientConfiguration', {
    'clientId': '******', 
    'openidIssuer': 'https://{your subdomain}.okta.com/oauth2/default',
    'clientHost': 'https://{kawa url}', 
    'offlineAccessScope': False, 
    'userIdField': 'email',
    'emailMustBeVerified': False,   
})

## 8. Configure connection to OPEN AI (optional)

This will allow KAWA to connect to OPEN AI APIs (Such as the assistant API and the chat API).


__Please note that the server needs to be restarted for the Open Ai configuration to be taken in account.__

In [ ]:
cmd.replace_configuration('OpenAiConfiguration', {
    'activated': True, 
    'openAiApiKey': 'sk-.........',
})

## 9. Configure API keys restrictions

This lets you define the maximum lifespan of the generated API keys. All API keys with a lifespan longer than this will be rejected.
The default value is: 365 days.


In [ ]:
cmd.replace_configuration('ApiKeyConfiguration', {
    'maximumLifespanInDays': 2
})